# conf.py

In [38]:
%%writefile manual/source/conf.py
# Configuration file for the Sphinx documentation builder.
#
# This file only contains a selection of the most common options. For a full
# list see the documentation:
# http://www.sphinx-doc.org/en/master/config

# -- Path setup --------------------------------------------------------------

# If extensions (or modules to document with autodoc) are in another directory,
# add these directories to sys.path here. If the directory is relative to the
# documentation root, use os.path.abspath to make it absolute, like shown here.
#
# import os
# import sys
# sys.path.insert(0, os.path.abspath('.'))

#import sphinx_rtd_theme


# -- Project information -----------------------------------------------------

project = 'The ulab book'
copyright = '2019-2020, Zoltán Vörös'
author = 'Zoltán Vörös'

# The full version, including alpha/beta/rc tags
release = '1.0.0'


# -- General configuration ---------------------------------------------------

# Add any Sphinx extension module names here, as strings. They can be
# extensions coming with Sphinx (named 'sphinx.ext.*') or your custom
# ones.
extensions = [
]

# Add any paths that contain templates here, relative to this directory.
templates_path = ['_templates']

# List of patterns, relative to source directory, that match files and
# directories to ignore when looking for source files.
# This pattern also affects html_static_path and html_extra_path.
exclude_patterns = []


# Add any paths that contain custom static files (such as style sheets) here,
# relative to this directory. They are copied after the builtin static files,
# so a file named "default.css" will overwrite the builtin "default.css".
html_static_path = ['_static']

master_doc = 'index'

author=u'Zoltán Vörös'
copyright=author
language='en'

latex_documents = [
(master_doc, 'the-ulab-book.tex', 'The $\mu$lab book',
'Zoltán Vörös', 'manual'),
]

Overwriting manual/source/conf.py


In [62]:
%%writefile manual/source/index.rst

.. ulab-manual documentation master file, created by
   sphinx-quickstart on Sat Oct 19 12:48:00 2019.
   You can adapt this file completely to your liking, but it should at least
   contain the root `toctree` directive.

Welcome to the ulab book!
=======================================


.. toctree::
   :maxdepth: 2
   :caption: Contents:

   ulab-intro
   ulab-ndarray
   ulab-approx
   ulab-compare
   ulab-fft
   ulab-filter
   ulab-linalg
   ulab-numerical
   ulab-poly
   ulab-vectorise
   ulab-programming

Indices and tables
==================

* :ref:`genindex`
* :ref:`modindex`
* :ref:`search`

Overwriting manual/source/index.rst


# Notebook conversion

In [58]:
import nbformat as nb
import nbformat.v4.nbbase as nb4
from nbconvert import RSTExporter

rstexporter = RSTExporter()
rstexporter.template_file = './templates/manual.tpl'

def convert_notebook(fn):
    source = nb.read(fn+'.ipynb', nb.NO_CONVERT)
    notebook = nb4.new_notebook()
    append_cell = False
    for cell in source['cells']:
        if append_cell:
            notebook.cells.append(cell)
        else:
            if cell.cell_type == 'markdown':
                if cell.source == '__END_OF_DEFS__':
                    append_cell = True
                    
    (rst, resources) = rstexporter.from_notebook_node(notebook)
    with open('./manual/source/' + fn + '.rst', 'w') as fout:
        fout.write(rst)

In [63]:
files = ['ulab-intro',
         'ulab-ndarray',
         'ulab-approx', 
         'ulab-compare',
         'ulab-fft',
         'ulab-filter',
         'ulab-linalg',
         'ulab-numerical',
         'ulab-poly',
         'ulab-vectorise',
         'ulab-programming']

In [64]:
for file in files:
    convert_notebook(file)

## Template

In [516]:
%%writefile ./templates/manual.tpl

{%- extends 'display_priority.tpl' -%}


{% block in_prompt %}
{% endblock in_prompt %}

{% block output_prompt %}
{% endblock output_prompt %}

{% block input scoped%}

{%- if cell.source.split('\n')[0].startswith('%%micropython') -%}
.. code::
        
{{ '\n'.join(['# code to be run in micropython'] + cell.source.strip().split('\n')[1:]) | indent}}

{%- else -%}
.. code::

{{ '\n'.join(['# code to be run in CPython\n'] + cell.source.strip().split('\n')) | indent}}
{%- endif -%}
{% endblock input %}

{% block error %}
::

{{ super() }}
{% endblock error %}

{% block traceback_line %}
{{ line | indent | strip_ansi }}
{% endblock traceback_line %}

{% block execute_result %}
{% block data_priority scoped %}
{{ super() }}
{% endblock %}
{% endblock execute_result %}

{% block stream %}
.. parsed-literal::

{{ output.text | indent }}
{% endblock stream %}

{% block data_svg %}
.. image:: {{ output.metadata.filenames['image/svg+xml'] | urlencode }}
{% endblock data_svg %}

{% block data_png %}
.. image:: {{ output.metadata.filenames['image/png'] | urlencode }}
{%- set width=output | get_metadata('width', 'image/png') -%}
{%- if width is not none %}
   :width: {{ width }}px
{%- endif %}
{%- set height=output | get_metadata('height', 'image/png') -%}
{%- if height is not none %}
   :height: {{ height }}px
{%- endif %}
{% endblock data_png %}

{% block data_jpg %}
.. image:: {{ output.metadata.filenames['image/jpeg'] | urlencode }}
{%- set width=output | get_metadata('width', 'image/jpeg') -%}
{%- if width is not none %}
   :width: {{ width }}px
{%- endif %}
{%- set height=output | get_metadata('height', 'image/jpeg') -%}
{%- if height is not none %}
   :height: {{ height }}px
{%- endif %}
{% endblock data_jpg %}

{% block data_markdown %}
{{ output.data['text/markdown'] | convert_pandoc("markdown", "rst") }}
{% endblock data_markdown %}

{% block data_latex %}
.. math::

{{ output.data['text/latex'] | strip_dollars | indent }}
{% endblock data_latex %}

{% block data_text scoped %}
.. parsed-literal::

{{ output.data['text/plain'] | indent }}
{% endblock data_text %}

{% block data_html scoped %}
.. raw:: html

{{ output.data['text/html'] | indent }}
{% endblock data_html %}

{% block markdowncell scoped %}
{{ cell.source | convert_pandoc("markdown", "rst") }}
{% endblock markdowncell %}

{%- block rawcell scoped -%}
{%- if cell.metadata.get('raw_mimetype', '').lower() in resources.get('raw_mimetypes', ['']) %}
{{cell.source}}
{% endif -%}
{%- endblock rawcell -%}

{% block headingcell scoped %}
{{ ("#" * cell.level + cell.source) | replace('\n', ' ') | convert_pandoc("markdown", "rst") }}
{% endblock headingcell %}

{% block unknowncell scoped %}
unknown type  {{cell.type}}
{% endblock unknowncell %}


Overwriting ./templates/manual.tpl
